In [ ]:
# Enable IPython autoreload for modules
%load_ext autoreload
%autoreload 2

import pandas as pd 
# Load configuration
data_dir = None # To silence Pylance; data_dir is defined by config.py
%run ../../config.py



In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

In [ ]:
from pathlib import Path
from subprocess import run

# URL for 2019 tract cartographic boundary file
tract_carto = "https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_tract_500k.zip"

tracts_fn = Path(tract_carto).name
tracts_path = Path(data_dir) / tracts_fn

if not tracts_path.exists():
    print(f"Downloading {tracts_fn}...")
    result = run(["wget", "-P", data_dir, tract_carto], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"Downloaded to {tracts_path}")
    else:
        print(f"Error downloading: {result.stderr}")
else:
    print(f"{tracts_fn} already exists at {tracts_path}")

# Load the tract shapefile

tracts_gdf = gpd.read_file(tracts_path)

tracts_gdf.head()

In [ ]:
# Get Census data - 2019 ACS 5-Year median household income by tract
# B19013_001E is median household income
# We'll get all tracts in all states

print("Fetching ACS 2019 5-Year median household income data by tract...")

# Get state FIPS codes first
states_url = "https://api.census.gov/data/2019/acs/acs5?get=NAME&for=state:*"
states_response = requests.get(states_url)
states_data = states_response.json()
state_fips = set([row[1] for row in states_data[1:]])  # Skip header

print(f"Found {len(state_fips)} states/territories")

# Fetch median income for all tracts in each state
all_tracts = []

for i, state_fips_code in enumerate(state_fips):  # Get all states
    try:
        url = f"https://api.census.gov/data/2019/acs/acs5?get=NAME,B19013_001E&for=tract:*&in=state:{state_fips_code}"
        response = requests.get(url)
        data = response.json()
        
        # Convert to dataframe
        df = pd.DataFrame(data[1:], columns=data[0])
        all_tracts.append(df)
        print(f"  State {state_fips_code}: {len(df)} tracts")
    except Exception as e:
        print(f"  Error fetching state {state_fips_code}: {e}")

# Combine all data
income_df = pd.concat(all_tracts, ignore_index=True)

# Clean up the data
income_df['median_income'] = pd.to_numeric(income_df['B19013_001E'], errors='coerce')
income_df['GEOID'] = income_df['state'] + income_df['county'] + income_df['tract']

income_df.head()


In [ ]:
# Join income data to tract geometries
print("Joining income data to tract shapefile...")

tracts_with_income = tracts_gdf.merge(
    income_df[['GEOID', 'median_income', 'NAME']], 
    on='GEOID', 
    how='left',
    suffixes=('_tract', '_income')
)

print(f"\nTracts with income data: {tracts_with_income['median_income'].notna().sum()}")
print(f"Tracts without income data: {tracts_with_income['median_income'].isna().sum()}")

# Filter to include only states that are in state_fips, then exclude Alaska (02) and Hawaii (15)
df = tracts_with_income_filtered = tracts_with_income[
    (tracts_with_income['STATEFP'].isin(state_fips)) & 
    (~tracts_with_income['STATEFP'].isin(['02', '15']))
]


df = df[df.median_income > 0]

df.describe()

In [ ]:
# Create map using GeoPandas plot method
fig, ax = plt.subplots(figsize=(30, 20))

# Use GeoPandas native plot without legend
df.plot(
    column='median_income',
    ax=ax,
    legend=False,
    cmap='viridis',
    missing_kwds={'color': 'lightgrey'}
)

ax.set_title('Median Household Income by Census Tract\n2019 ACS 5-Year Estimates', 
             fontsize=20, fontweight='bold')
ax.set_axis_off()

plt.tight_layout()
plt.show()


In [ ]:
df.median_income.hist(bins=100)

In [ ]:
df[df.median_income>0].median_income.describe()

In [ ]:
# Save joined tracts with income to Parquet in data_dir
import os

output_path = os.path.join(data_dir, "tracts_with_income_2019.parquet")
df.to_parquet(output_path)
print(f"Saved {len(df)} rows to {output_path}")